In [5]:
import spacy
import pandas as pd
import numpy as np

import Modules.solrhandler as sh
import Modules.clusterer as cls
import Modules.topicdeterminator as td
import Modules.chatbot as cb

In [ ]:
#query = "anmeldung"
#target_service = 120686

Evaluierungspipeline

In [6]:
# Import eval.csv
import os
file = os.path.realpath("../../Evaluierung/eval.csv")
eval = pd.read_csv(file, sep = ";")
eval = eval.head(5)
eval

,documentId,file,dialogId,initialQuestion,suggestionCount,directlyFound,name
0,120697,20220131--quantEvalAllExcel.xlsx,4,Ummeldung,6,True,Änderung/Wechsel der Hauptwohnung
1,120686,20220131--quantEvalAllExcel.xlsx,4,Ummelden,6,True,Anmeldung einer Wohnung
2,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...
3,329421,20220131--quantEvalAllExcel.xlsx,19,Quarantäne Krankschreibung vom Arzt,6,True,Erstattung nach Infektionsschutzgesetz bei Tät...
4,120914,20220131--quantEvalAllExcel.xlsx,23,Termin für Zulassung Gebrauchtwagen,6,True,Zulassung eines Fahrzeuges mit auswärtigem Ken...


Funktionen

In [7]:
def findCorrectAnswer(chatbot,targetService):

    # Find Target Cluster
    clusteredColumn = chatbot.clusterer.getClusteredColumn()
    service = chatbot.df.loc[chatbot.df["id"]==str(targetService)]
    if len(service[clusteredColumn].values) == 0:
        return None

    targetCluster = service[clusteredColumn].values[0]

    if targetCluster == chatbot.getSelectedClusterForQuestion():
        return True

    else:
        return False

def getRank(chatbot, targetService):
    temp = chatbot.df#.reset_index()
    rank = temp.index[temp["id"]==str(targetService)].values[0]
    return rank

Logging

In [8]:
class Logger:
    
    def __init__(self, chatbot, eval_df):
        self.liste = []
        self.chatbot = chatbot
        self.eval_df = eval_df
    
    def getDataFrame(self): # TODO
        pass
    
    def log(self, t, row, answer = None): 
        target_service = row["documentId"]
        
        i = row["dialogId"]
        file = row["file"]
        if answer is not None:

            
            nResults = len(self.chatbot.df.index)
            question = self.chatbot.generateQuestion()
            answer = answer

            rank = getRank(self.chatbot,target_service)# getRank(target_service)
            
            row = (file, i, t, nResults, question, answer, rank)
        else:
            #nInitialResults = len(chatbot.df.index)
            #initialRank = np.nan
            row = (file, i, t, None, None, None, None)
        self.liste.append(row)
        #print(self.liste)

Execution

In [9]:
# Initialisierung Komponenten Chatbot
solrhandler = sh.SolrHandler(20)
clusterer = cls.Clusterer()
tpc_dterminator = td.TopicDeterminator()
chatbot = cb.Chatbot(solrhandler,clusterer,tpc_dterminator)

lg = Logger(chatbot,eval)   

for (dialogue_id, row) in eval.iterrows():#np.arange(0,len(eval.index)-1):#[0,1,2,3,4]
    
    # fun
    query = row["initialQuestion"]#"Ummeldung"
    target_service = row["documentId"]
    
    # Log
    
    # TODO ERROR???
    try:
        chatbot.initialQuery(query)
    except Exception as e:
        if(str(e) == 'response'):
            next
        elif(str(e) == 'no solr output'):
            next
    
    # TODO
    t = 0
    answer = findCorrectAnswer(chatbot, target_service)
    lg.log(t, row, answer)
    if answer is None:
        continue
    
    
    while not chatbot.refineResultset(answer):
        # log
        t += 1
        
        answer = findCorrectAnswer(chatbot, target_service)
        if answer is None: 
            break
        lg.log(t, row,answer)
    t+=1
    #lg.log(i,t,answer)
    
        


C:\Users\morit\anaconda3\envs\chatbot\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\morit\anaconda3\envs\chatbot\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\morit\anaconda3\envs\chatbot\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\morit\anaconda3\envs\chatbot\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarni

In [10]:
#query
len(solrhandler.get_df_from_query(query))

20

In [11]:
log_df = pd.DataFrame(lg.liste, columns=["file","dialogId","t","nResults", "question", "answer", "rank"])
log_df

,file,dialogId,t,nResults,question,answer,rank
0,20220131--quantEvalAllExcel.xlsx,4,0,7.0,Geht es bei ihrem Anliegen um zulassung?,True,2.0
1,20220131--quantEvalAllExcel.xlsx,4,0,NaN,None,None,NaN
2,20220131--quantEvalAllExcel.xlsx,19,0,13.0,Geht es bei ihrem Anliegen um approbation?,False,3.0
3,20220131--quantEvalAllExcel.xlsx,19,1,6.0,Geht es bei ihrem Anliegen um sars?,True,3.0
4,20220131--quantEvalAllExcel.xlsx,19,0,10.0,Geht es bei ihrem Anliegen um approbation?,False,0.0
5,20220131--quantEvalAllExcel.xlsx,19,1,3.0,Geht es bei ihrem Anliegen um quarantäne?,True,0.0
6,20220131--quantEvalAllExcel.xlsx,23,0,20.0,Geht es bei ihrem Anliegen um kfz?,True,6.0


Log Dataset Corrections and join with eval dataset

In [12]:
log_df["rank"] = log_df["rank"]+1

In [13]:
test = pd.merge(eval, log_df, on=["file", "dialogId"], how="inner")
test

,documentId,file,dialogId,initialQuestion,suggestionCount,directlyFound,name,t,nResults,question,answer,rank
0,120697,20220131--quantEvalAllExcel.xlsx,4,Ummeldung,6,True,Änderung/Wechsel der Hauptwohnung,0,7.0,Geht es bei ihrem Anliegen um zulassung?,True,3.0
1,120697,20220131--quantEvalAllExcel.xlsx,4,Ummeldung,6,True,Änderung/Wechsel der Hauptwohnung,0,NaN,None,None,NaN
2,120686,20220131--quantEvalAllExcel.xlsx,4,Ummelden,6,True,Anmeldung einer Wohnung,0,7.0,Geht es bei ihrem Anliegen um zulassung?,True,3.0
3,120686,20220131--quantEvalAllExcel.xlsx,4,Ummelden,6,True,Anmeldung einer Wohnung,0,NaN,None,None,NaN
4,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...,0,13.0,Geht es bei ihrem Anliegen um approbation?,False,4.0
5,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...,1,6.0,Geht es bei ihrem Anliegen um sars?,True,4.0
6,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...,0,10.0,Geht es bei ihrem Anliegen um approbation?,False,1.0
7,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...,1,3.0,Geht es bei ihrem Anliegen um quarantäne?,True,1.0
8,329421,20220131--quantEvalAllExcel.xlsx,19,Quarantäne Krankschreibung vom Arzt,6,True,Erstattung nach Infektionsschutzgesetz bei Tät...,0,13.0,Geht es bei ihrem Anliegen um approbation?,False,4.0
9,329421,20220131--quantEvalAllExcel.xlsx,19,Quarantäne Krankschreibung vom Arzt,6,True,Erstattung nach Infektionsschutzgesetz bei Tät...,1,6.0,Geht es bei ihrem Anliegen um sars?,True,4.0
